####   D (англ. delete) — удалить,
####   I (англ. insert) — вставить,
####   R (replace) — заменить,
####   M (match) — совпадение.

####   w(a, а) = 0
####   w(a, b) = по формуле при a≠b - цена замены символа a на символ b
####   w(ε, b) = 1 - цена вставки символа b
####   w(a, ε) = 1 - цена удаления символа a

In [1]:
import numpy as np
import pandas as pd
import itertools
import copy

# the Levenshtein distance

Таблица всех звуков, преобразую ее.

In [2]:
data = pd.read_csv('fon_table.csv/Лист 1-Tаблица 1.csv')
data = data.set_index('name')
data = data.transpose()
data = data.fillna('None')
data = pd.DataFrame(data)

Создаю будущую матрицу расстояний.

In [3]:
dist_matrix = pd.DataFrame(np.zeros((54, 54)))#, dtype=int))

In [4]:
column_names = {}
for i in range(0, 54):
    column_names[i] = data.columns.tolist()[i]

In [5]:
dist_matrix = dist_matrix.rename(index=str, columns=column_names)
dist_matrix['index'] = data.columns.tolist()
dist_matrix = dist_matrix.set_index('index')

Заполняю клеточки в таблице результатами формулы

1 - S_rows/ C_rows + Unc_rows*2 

In [6]:
comobinations = itertools.combinations_with_replacement(data.columns, 2) # - все возможные комбинации

In [7]:
for i in comobinations:

    a1 = pd.get_dummies(data[i[0]]).drop(['None'], axis=1)
    a = a1.loc[~(a1==0).all(axis=1)]
    b1 = pd.get_dummies(data[i[1]]).drop(['None'], axis=1)
    b = b1.loc[~(b1==0).all(axis=1)]
    
    dif_a = a.index.difference(b.index)
    similar = a.index.difference(dif_a)
    
    d = a.loc[similar] + b.loc[similar]
    s_rows = 0
    counts = d['+'].value_counts()

    if 0 in counts:
        s_rows += counts[0]
    if 2 in counts:
        s_rows += counts[2]
    
                        # common                uncommon
    result = 1 - s_rows/(len(similar) + (a.shape[0] + b.shape[0] - 2*len(similar)))# * 2)

    dist_matrix[i[0]][i[1]] = result
    dist_matrix[i[1]][i[0]] = result


Итог

In [8]:
dist_matrix.head()

,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
index,,,,,,,,,,,,,,,,,,,,,
и,0.000,0.125,0.125,0.250,0.250,0.375,0.250,0.125,0.375,0.375,...,0.785714,0.846154,1.000000,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571
ы,0.125,0.000,0.250,0.125,0.375,0.250,0.125,0.250,0.250,0.250,...,0.857143,0.923077,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571,0.857143
е,0.125,0.250,0.000,0.125,0.125,0.250,0.375,0.250,0.250,0.250,...,0.785714,0.846154,1.000000,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571
э,0.250,0.125,0.125,0.000,0.250,0.125,0.250,0.375,0.125,0.125,...,0.857143,0.923077,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571,0.857143
ä,0.250,0.375,0.125,0.250,0.000,0.125,0.500,0.375,0.375,0.375,...,0.785714,0.846154,1.000000,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571


##### Расстояние Левенштейна.


Я разобралась с визуализацией расстояния (http://www.let.rug.nl/kleiweg/lev/) и на осонове этой идеи сделала свою фунцию: 

##### Новая версия

In [9]:
def lev_distance(a, b, dist_matrix):
    
    # Первыми - строчки 
    # столбики - слово b
    
    dis = np.zeros((len(a) + 1, len(b) + 1))
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row - 1, col] + 1
        
  
        elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
            dis[row, col] = dis[row - 3][col - 3] + 1
            
        
        else:
            dis[row, col] = np.min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + dist_matrix[a[row-1]][b[col-1]], # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1

    return dis[len(a), len(b)]

In [11]:
lev_distance('мила', 'мил', dist_matrix)

[[0. 1. 2. 3.]
 [1. 0. 1. 2.]
 [2. 1. 0. 1.]
 [3. 2. 1. 0.]
 [4. 3. 2. 1.]]


1.0

In [12]:
lev_distance('порт', 'спирт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 1.375      2.375      3.375     ]
 [3.         2.27272727 1.72727273 2.06320346 1.375      2.375     ]
 [4.         3.18181818 2.36363636 2.72727273 2.375      1.375     ]]


1.375

In [154]:
lev_distance('порт', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 1.         2.         3.        ]
 [3.         2.27272727 1.72727273 1.99177489 1.         2.        ]
 [4.         3.18181818 2.36363636 2.66060606 2.         1.        ]]


1.0

In [155]:
lev_distance('прот', 'порт', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.78571429 1.         2.        ]
 [3.         2.         1.         1.         1.93333333]
 [4.         3.         2.         1.36363636 1.        ]]


1.0

In [156]:
lev_distance('прот', 'пирт', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.85714286 1.         2.        ]
 [3.         2.         1.375      1.64285714 1.93333333]
 [4.         3.         2.375      1.73863636 1.64285714]]


1.6428571428571428

In [157]:
lev_distance('прот', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 0.72727273 1.72727273 2.         3.        ]
 [3.         2.27272727 1.72727273 0.72727273 2.         2.93333333]
 [4.         3.18181818 2.36363636 1.72727273 1.09090909 2.        ]]


2.0

In [173]:
lev_distance('мил', 'мила', dist_matrix)

[[0. 1. 2. 3. 4.]
 [1. 0. 1. 2. 3.]
 [2. 1. 0. 1. 2.]
 [3. 2. 1. 0. 1.]]


1.0

In [10]:
import collections
from collections import defaultdict

In [11]:
def sort_words(words, word, dist_matrix, top=10):
    
    answer = collections.defaultdict(list)
    
    for item in words:

        distance = lev_distance(word, item, dist_matrix)
        answer[item].append(distance)
        
    return sorted(answer, key=lambda n: answer[n])[:top]

In [12]:
words = ['спорт', 'торт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word, dist_matrix)

['торт', 'парт', 'прот', 'спорт', 'кипр', 'спирт']

In [13]:
words = ['мам', 'мамы',  'маму', 'мыла', 'папа', 'мышь', 'смама'] 
word = 'мама'
sort_words(words, word, dist_matrix)

['мамы', 'маму', 'папа', 'мыла', 'смама', 'мышь', 'мам']

Проблема решена.

## Фонетические изменения

#### ГЛАСНЫЕ

In [14]:
vows = pd.read_csv('vowels.csv', sep=';')
vows = vows.set_index('Unnamed: 0')
vows = vows.reindex(vows.index.rename('name'))

In [15]:
vows

,#,v2_hard,v2_soft,v1_hard,v1_sh,v1_soft,V,vn_hard,vn_soft
name,,,,,,,,,
а,а,ъ,ь,а,a,и,а,ъ,ь
я,а,ъ,ь,а,a,и,а,ъ,ь
о,а,ъ,ь,а,ы,и,о,ъ,ь
ё,а,ъ,ь,а,ы,и,о,ъ,ь
у,у,у,у,у,у,у,у,у,у
ю,у,у,у,у,у,у,у,у,у
е,и,ъ,ь,ы,ы,и,е,ъ,ь
э,и,ъ,ь,ы,ы,NaN,э,ъ,ь
и,и,NaN,ь,NaN,NaN,и,и,NaN,ь


#### СОГЛАСНЫЕ

In [16]:
cons = pd.read_csv('cons.csv', sep=';')
cons = cons.set_index('Unnamed: 0')
cons = cons.reindex(cons.index.rename('name'))
cons

,soft,hard,special,voiced,no_voice
name,,,,,
б,б’,б,NaN,б,п
в,в’,в,NaN,в,ф
г,г’,г,NaN,г,к
д,д’,д,NaN,д,т
ж,NaN,ж,ж,ж,ш
з,з’,з,NaN,з,с
й,j,NaN,ṷ,j,j
к,к’,к,NaN,г,к
л,л’,л,NaN,л,л


Табилца, которая помогает учесть ассимиляцию

In [104]:
aff = pd.read_csv('aff.csv', sep=';')
aff = aff.set_index('Unnamed: 0')
aff = aff.reindex(aff.index.rename('name'))
aff

,с,ш,щ,ч,ж
name,,,,,
с,-,ш,ш’,ш’,ж
сс,-,-,-,ш’,-
ст,-,-,-,ш’,-
з,-,ш,ш’,ш’,-
зд,-,-,ш’,ш’,-
т,ц,ч,-,-,-
ть,ц,-,-,-,-
д,ц,ч,-,-,-
ж,-,-,-,ш’,ж


### ТАБЛИЦА УДАРЕНИЙ

In [18]:
st_words = pd.read_csv('ru_reversed.csv', sep='\t')

In [19]:
new = st_words['word'].str.replace('(а́|А́)', 'а')
new = new.str.replace('(у́|У́)', 'у')
new = new.str.replace('(и́|И́)', 'и')
new = new.str.replace('(о́|О́)', 'о')
new = new.str.replace('(е́|Е́)', 'е')
new = new.str.replace('(э́|Э́)', 'э')
new = new.str.replace('(ы́|Ы́)', 'ы')
new = new.str.replace('(ю́|Ю́)', 'ю')
new = new.str.replace('(я́|Я́)', 'я')

In [20]:
new = new.str.lower()

In [21]:
st_words['norm_form'] =  new 

In [22]:
st_words = st_words.set_index('norm_form')

In [23]:
st_words.head()

,word,stressed_s
norm_form,,
а,а́,1
а,а́,1
а,а́,1
а.,а́.,1
аа,а́а,2


## ТРАНСКРИПЦИЯ

Функция, которая ищет заданное слово в словаре с ударениями. Возращает номер гласной, на которой стоит ударение. Чтобы не использовать дополнительную память, сделала из него генератор. Если слова в словаре нет, то слову пока(!) приписывается ударение на первый гласный, чтобы обезопасить от ошибок (например, если в слове всего одна гласная, а я бы вернула вторую)

In [24]:
def stressed(word, st_words):
    
    try:
        a = st_words.loc[word]['stressed_s'].tolist()
   
        if isinstance(a, list):
            yield from a
        else:
            yield from [a]
    
    except KeyError:
        yield from [1]

Главная идея такая: идти с конца слова и преобразовывать каждую букву в звук.

Эта функция преобразует гласные буквы в соотвествии с таблицей. У меня есть таблица с соответсвиями гласных и позиции в слове. Функция возвращает нужный вариант и, если требуется, делает дополнительные преобразования: указание мягкости для следующего согласного, добавление j, если это ударный слог, и тд.

In [ ]:
def ans_stressed(letter, next_letter, index, length ):
    
    if letter in ('э', 'е') and index + 1 <= length-1 and letters[index+1] in ('ш', 'ж', 'ц'):
                letter = 'э'
    
        ans = j(letter, index, length, vows, letters)
        
    if letter in ('е', 'ё', 'и', 'ю', 'я'):
        
        if index + 1 <= length-1 and letters[index+1] not in vows.index:
            soft = True
    return 

Изменения гласных

In [25]:
def due_to_vow_table(vow_n, i, letter, index, length, vows, letters, soft):
    
    #ans = letter

    if vow_n == i:
        
        if letter in ('э', 'е') and index + 1 <= length-1 and letters[index+1] in ('ш', 'ж', 'ц'):
                letter = 'э'
    
        ans = j(letter, index, length, vows, letters)
        
        if letter in ('е', 'ё', 'и', 'ю', 'я'):
            if index + 1 <= length-1 and letters[index+1] not in vows.index:
                soft = True
    
                    
    elif vow_n == i + 1: # первый предударный
        
        if index + 1 <= length-1 and letters[index+1] in ('ц', 'ж', 'ш'):
            ans = vows.loc[letter]['v1_sh']
        
        elif index + 1 <= length-1 and letters[index+1] in vows.index:
            if letter in ('е', 'ё', 'и', 'ю', 'я'):
                ans = vows.loc[letter]['v1_soft']
                
            else:
                ans = vows.loc[letter]['v1_hard']
            
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans = vows.loc[letter]['v1_soft']
            
            if index + 1 <= length-1 and letters[index+1] not in vows.index:
                soft = True
        else:
            ans = vows.loc[letter]['v1_hard']

                    
    elif vow_n >= i + 2: # второй предударный
        
        if letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans = vows.loc[letter]['v2_soft']
            
            if index + 1 <= length-1 and letters[index+1] not in vows.index:
                soft = True
                
        elif index + 1 <= length-1 and letters[index+1] in vows.index:
            if letter in ('е', 'ё', 'и', 'ю', 'я'):
                ans = vows.loc[letter]['v1_soft']
                
            else:
                ans = vows.loc[letter]['v1_hard']

        else:
            ans = vows.loc[letter]['v2_hard']
        
                    
    elif vow_n < i: # заударные 
        
        if letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans = vows.loc[letter]['vn_soft']
            
            if index + 1 <= length-1 and letters[index+1] not in vows.index:
                soft = True
        else:
            ans = vows.loc[letter]['vn_hard']
    
                
    elif index == length - 1:  # начало
        ans = vows.loc[letter]['#']
        
    return ans, soft

Функция, которая вставляет j

In [26]:
def j(letter, index, length, vows, letters):
    
    ans = vows.loc[letter]['V']
    
    if letter in ('ю', 'е', 'ё', 'я'):
        
        if index == length - 1:
            ans = 'j' + ans
    
        elif index + 1 <= length-1 and letters[index+1] in vows.index:
            ans = 'j' + ans
        
        elif index + 1 <= length-1 and letters[index+1] in ('ь', 'ъ'):
            ans = 'j' + ans
    
    elif letter in ('и', 'о') and index + 1 <= length-1 and letters[index+1] == 'ь':
        ans = 'j' + ans
    
    return ans

Изменения согласных

In [114]:
def cons_tranformer(letter, no_voice, soft, voice, index, data, cons, length, letters, gap, answer, vow_n, i):

    if gap:
        return '', no_voice, soft, voice, gap
    
    if letter in ('ч', 'ш', 'щ', 'ж'):
        ans = cons.loc[letter]['special']
    else:
        ans = letter 

    no_voice = no_voice
    soft = soft
    voice = voice

    if index == 0 or no_voice == True:
        ans = cons.loc[letter]['no_voice']
        no_voice == False
    
    if voice == True:
        ans = cons.loc[letter]['voiced']
        voice == False
    
    if soft == True and ans not in ('ч’', 'ш’'):
        ans = cons.loc[ans]['soft']
        soft = False
        
    if index + 1 <= length-1 and letters[index+1] in cons.index:

        if letter in aff.columns:
        
            if index + 2 <= length-1 and letters[index+2] in cons.index:

                if letters[index+2] + letters[index+1] in aff.index:
                    ans2 = aff.loc[letters[index+2] + letters[index+1]][letter]
                    if ans2 != '-':
                        gap = True
                        
                        
                        # прописать изменение гласных
                        
                        # if len(answer) != 0 and index - 1 >= 0 and letters[index-1] in vows.index:
                            
                            # if vow_n == i:
                            #    answer[-1] = 
                        
                        return ans2, no_voice, soft, voice, gap
        
            elif letters[index+1] in aff.index:
                ans2 = aff.loc[letters[index+1]][letter]
                if ans2 != '-':
                    gap = True
                    
                    # прописать изменение гласных
                            
                    return ans2, no_voice, soft, voice, gap
                
        
        if data[ans]['son'] == '-' and ans not in ('в', "в’") and data[ans]['vcd'] == '+':
            voice = True
  
        elif letters[index+1] not in ('в', "в’") and data[ans]['vcd'] == '-':
            
            next_ = letters[index+1]
            if next_ in ('й', 'ч', 'ш', 'щ', 'ж'):
                next_ = cons.loc[next_]['special']
     
            if data[next_]['son'] == '-':
                no_voice = True
            

    if index-1 >= 0:
        
        if letters[index-1] == 'к' and letter == 'г':
            ans = 'х'
        
        elif answer[index-1] == 'т' and ans == "ч’":
            ans = 'ш'
        
        elif answer[index-1] == ans:
            ans = ''
        

    return ans, no_voice, soft, voice, gap

In [115]:
import copy

def correct_cons(data, letter, new):
    
    if (data[letter] == new).all():
        return letter

    for i in data.columns:
        if (new == data[i]).all():
            return i
    return ''

Так как в словаре с ударениями могут быть повторения (например, слова одинаково пишутся, но ударение на разных слогах, или просто в словаре есть повторения), то я рассматриваю все варианты. 

In [108]:
words = ['датский', 'обходится', 'здщ', 'смотреться', 'прочти', 'рагк', 'бадтут', 'матд', 'массаж', 'рассвет', 
         'код', 'свой', 'съесть', 'синяя', 'зоозащитник', 'садовод', 'типовой', 'лесок', 
         'лес', 'мёл', 'поэт', 'поет', 'поёт', 'елка',
         'еду', 'наелся', 'жест', 'цех', 'шест', 'те', 'эта', 'пил', 'пел', 'бульон', 'май', 'цены', 'цена']


In [116]:
for word in words:
    
    length = len(word)
    letters = list(word)[::-1]

    for i in stressed(word, st_words):
        
        answer = []
        vow_n = 0
        soft = False
        voice = False
        no_voice = False
        gap = False
        
        for index, letter in enumerate(letters):
            
            if letter in vows.index:
                
                voice = False
                no_voice = False
                gap = False
         
                vow_n += 1
            
                ans, soft = due_to_vow_table(vow_n, i, letter, index, length, vows, letters, soft)
            
            
            elif letter == 'ь' or letter == 'ъ': 
                ans = ''
                
                if letter == 'ь':
                    
                    if index - 1 >= 0 and letters[index-1] in aff.columns:
                        
                        if index + 1 <= length-1 and letters[index+1] + letter in aff.index: 
                            
                            answer[-1] = ''
                            ans = aff.loc[letters[index+1] + letter][letters[index-1]]
                            gap = True
                    
                    else:
                        soft = True
                
                if letter == 'ъ':
                    soft = False
            
            else:
                
                if letter == 'й':
                    ans = cons.loc[letter]['special']
                    
                else:
                    ans, no_voice, soft, voice, gap = cons_tranformer(letter, no_voice, soft, voice, index, data, cons, length, letters, gap, answer, vow_n, i)

            answer.append(ans)

        print(word + ': [' + ''.join(answer[::-1]) + ']')
                    
            

датский: [дацк’ьṷ]
обходится: [апход’ьць]
обходится: [апход’ьць]
здщ: [ш’]
смотреться: [сматр’ець]
смотреться: [сматр’ець]
прочти: [прач’т’и]
рагк: [рахк]
бадтут: [батут]
матд: [мат]
массаж: [масаш]
рассвет: [расв’ет]
код: [кот]
код: [кот]
свой: [своṷ]
съесть: [сjест’]
синяя: [с’ин’ьь]
зоозащитник: [зъаз’ъш’итн’ик]
садовод: [съдавот]
типовой: [т’ьпавоṷ]
лесок: [л’есък]
лесок: [л’исок]
лес: [л’ес]
лес: [л’ес]
мёл: [м’ол]
поэт: [паэт]
поет: [паjет]
поёт: [паjот]
елка: [илка]
еду: [jеду]
еду: [иду]
наелся: [нъилс’а]
жест: [жэст]
цех: [цэх]
шест: [шэст]
те: [т’е]
те: [т’е]
эта: [этъ]
эта: [этъ]
пил: [п’ил]
пил: [п’ил]
пел: [п’ел]
бульон: [бул’jон]
май: [маṷ]
май: [маṷ]
май: [маṷ]
цены: [цэнъ]
цены: [цыны]
цена: [цына]


То, что нужно учесть

In [88]:
Ассимиляция 

сш- ш
зш- ш
зж- ж
сж- ж
жж - ж
тс - ц
тьс - ц
дс - ц
сщ - ш'
зщ - ш'
тш - ч
дш - ч
здщ - ш'
зч - ш'
сч - ш'
ссч - ш'
шч - ш'
жч - ш'
стч - ш'
здч - ш'

SyntaxError: EOL while scanning string literal (<ipython-input-88-52117a05f754>, line 11)

In [ ]:
две одинаковые - одна 1  + 
звонкая, глухая (одинаковый) - глухая 1  +
глухая, звонкая (одинаковый) - звонкая 1  +

In [ ]:
гк -> хк + 
ч'т - [шт]

## СТАРЫЕ НАРАБОТКИ

Здесь возникает некоторая проблема с перестановками. Так как этот алгорит сравнивает строчки линейно, то есть П и С, П и СП, П и СПО и тд. Моя идея заключалась в том, что замена происходит только тогда, когда буквы (сейчас я рассматриваю имеено буквы) находятся на одной и той же позиции, то есть П и С, ПО и СП и тд, то есть я сравниваю буквы именно тогда.
Здесь алгоритм не учитывает той ситуации, что две буквы могут совпадать и быть при этом не на одной и той же позиции, из-за этого получается так, что разница между спортом и портом 0.27, хотя должно быть 1. 

In [513]:
lev_distance1('порт', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.20606061 0.27272727 1.27272727 2.27272727]
 [3.         2.27272727 2.20606061 1.05844156 0.27272727 1.27272727]
 [4.         3.27272727 3.20606061 2.05844156 0.63636364 0.27272727]]


0.2727272727272727

In [520]:
lev_distance1('порт', 'спирт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.20606061 1.27272727 2.27272727 3.27272727]
 [3.         2.27272727 2.20606061 2.06320346 1.27272727 2.27272727]
 [4.         3.27272727 3.20606061 3.06320346 1.63636364 1.27272727]]


1.2727272727272727

Здесь я постаралась решить эту проблему, добавив условие, которое, в случае, когда буквы одинаковые но стоят на разных позициях в слове, ставит результатом их сравнения разницу в позициях. Если рассматривать случай порта и спорта, это равно 1, то есть, чтобы превраить СП в П, нужно удалить одну букву. 

In [516]:
def lev_distance2(a, b, dist_matrix):
    
    # Первыми - строчки 
    # столбики - слово b
    
    dis = np.zeros((len(a) + 1, len(b) + 1))
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row-1, col] + 1
        
        else:
   
            dis[row, col] = np.min([dis[row, col - 1],  # левый 
                                    dis[row - 1, col - 1], # диаг               
                                    dis[row - 1, col]]) # верхний
    
            if row == col:
                dis[row, col] += dist_matrix[a[row-1]][b[col-1]] # замена
                
            elif a[row-1] == b[col-1] and row != col:
                dis[row, col] = abs(row - col)  # совпадают на разных позициях
               
            elif a[row-1] != b[col-1]:
                dis[row, col] += 1 # все остальное
            
            #if dis[row, col] == dis[row, col-1]:
               # dis[row, col] = 1
               # dis[row, col-1] = 1
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1
    print(dis)
    return dis[len(a), len(b)]

In [517]:
lev_distance2('порт', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 1.         2.         3.        ]
 [3.         2.27272727 2.20606061 1.78571429 1.         2.        ]
 [4.         3.27272727 3.20606061 2.78571429 1.36363636 1.        ]]


1.0

Однако из-за этого теряется противопоставление и - о, которое хорошо разграничивалось в предыдущей формуле.

In [519]:
lev_distance2('порт', 'спирт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 2.         3.         4.        ]
 [3.         2.27272727 2.20606061 2.06320346 1.         2.        ]
 [4.         3.27272727 3.20606061 3.06320346 1.36363636 1.        ]]


1.0

Тогда я решила попробовать внедрить способ, который использовала в предыдущих версиях - сравнивать линейные комбинации букв слова из словаря со словом, заданным пользователем. То есть я рассмотриваю комбинации спор, порт. Затем выбираю наименьшее расстояние до порта и прибавляю разницу в длине (= кол-во удалений). Расстояние я здесь считаю по lev_distance1

In [44]:
def sort_words(words, word, dist_matrix):
    
    result = {}
    
    for item in words:
        
        if len(item) != len(word):
            c = 0
            if len(item) < len(word):
                item, word = word, item
                c += 1

            diff = len(item) - len(word)
            l = len(word)
            var = len(item) - len(word)
    
            for i in range(diff + 1):
    
                dis = lev_distance(word, item[i:l], dist_matrix)
                if dis < var:
                    var = dis   
                l += 1
        
            dist = dis + diff
             
            if c != 0:
                item, word = word, item
        
        else:
            dist = lev_distance(word, item, dist_matrix)
        
        if dist not in result:
            result[dist] = [item]
        else:
            result[dist].append(item)

    return result

In [522]:
words = ['спирт']
word = 'порт'
sort_words(words, word, dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         1.         1.         2.         3.        ]
 [1.         1.         1.93333333 2.         3.        ]
 [1.         1.         2.         2.79047619 2.        ]
 [1.         1.         2.         3.         2.36363636]]
[[0.    1.    2.    3.    4.   ]
 [1.    0.    1.    2.    3.   ]
 [2.    1.    0.375 1.375 2.375]
 [3.    2.    1.375 0.375 1.375]
 [4.    3.    2.375 1.375 0.375]]


{1.375: ['спирт']}

In [523]:
words = ['спорт']
word = 'порт'
sort_words(words, word, dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         1.         1.         2.         3.        ]
 [1.         1.         1.93333333 1.         2.        ]
 [1.         1.         2.         1.78571429 1.        ]
 [1.         1.         2.         2.78571429 1.36363636]]
[[0. 1. 2. 3. 4.]
 [1. 0. 1. 2. 3.]
 [2. 1. 0. 1. 2.]
 [3. 2. 1. 0. 1.]
 [4. 3. 2. 1. 0.]]


{1.0: ['спорт']}

In [495]:
words = ['спорт', 'торт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word, dist_matrix)

{0.09090909090909094: ['торт'],
 0.25: ['парт'],
 1.0: ['спорт'],
 1.2840909090909092: ['кипр'],
 1.375: ['спирт'],
 1.5714285714285714: ['прот']}

In [496]:
words = ['спорт', 'прт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word, dist_matrix)

{0.25: ['парт'],
 1.0: ['спорт'],
 1.2840909090909092: ['кипр'],
 1.375: ['спирт'],
 1.5714285714285714: ['прот'],
 1.9333333333333333: ['прт']}

In [497]:
words = ['мам', 'мамы',  'маму', 'мыла', 'папа', 'мышь', 'смама'] 
word = 'мама'
sort_words(words, word, dist_matrix)

{0.25: ['мамы'],
 0.375: ['маму'],
 0.5454545454545454: ['папа'],
 0.85: ['мыла'],
 1.0: ['смама'],
 1.2613636363636362: ['мышь'],
 2.7142857142857144: ['мам']}

С этой точки зрения проблема решена, однако есть переставноки вида порт и прот. 

In [531]:
lev_distance2('порт', 'прот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.78571429 1.         2.        ]
 [3.         2.         1.         1.57142857 2.        ]
 [4.         3.         2.         2.         1.57142857]]


1.5714285714285714

In [532]:
lev_distance1('порт', 'прот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.78571429 0.78571429 1.78571429]
 [3.         2.         0.78571429 1.57142857 1.78571429]
 [4.         3.         1.78571429 1.78571429 1.57142857]]


1.5714285714285714

Самое простое решение, которое я вижу - рассматривать расстояние вообще ото всех возможных комбинаций длины заданного слова, однако это сулит большим временем работы.

Сейчас я пытаюсь, что-то придумать пути с изменение результатов в матрице. Например, если у нас есть последовательность из двух одинаковых чисел, то, возможно, в этом месте есть перестановка, однако пока не очень понятно, как сохранить разницу между предшедмтвующими звуками

In [534]:
lev_distance2('порт', 'мрот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.05844156 1.         2.        ]
 [3.         2.27272727 1.         1.78571429 2.        ]
 [4.         3.27272727 2.         2.         1.78571429]]


1.7857142857142856

In [535]:
lev_distance1('порт', 'мрот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.05844156 1.05844156 2.05844156]
 [3.         2.27272727 1.05844156 1.84415584 2.05844156]
 [4.         3.27272727 2.05844156 2.05844156 1.84415584]]


1.844155844155844

In [ ]:
cons = {'б': ['п', ], 'г': ['к', 'г’'], 
        'к': ['к'], 'д': ['т'], 
        'т': ['т'], 'з': ['с'], 
        'с': ['с'], 'ж': ['ш'], 
        'п': ['п'], 'й': ['й'], 
        'м': ['м'], 'л': ['л'], 
        'р': ['р'], 'в': ['ф'], 
        'ф': ['ф'], 'х': ['х'], 
        'щ': ['щ'],'ш': ['ш'], 
        'ц': ['ц'], 'нг': ['ŋ']}